In [1]:
# !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split

from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
# from finrl.env.EnvMultipleStock_train import StockEnvTrain
# from finrl.env.EnvMultipleStock_trade import StockEnvTrade

from finrl.agents.stablebaselines3.models import DRLAgent

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
# from finrl.trade.backtest import BackTestStats, BaselineStats, BackTestPlot, backtest_strat, baseline_strat

c:\Users\aminh\anaconda3\envs\py38_stocks\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\aminh\anaconda3\envs\py38_stocks\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [3]:
# # Data and preprocessing
# TRAIN_START_DATE = '2009-04-01'
# TRAIN_END_DATE = '2021-01-01'
# TEST_START_DATE = '2021-01-01'
# TEST_END_DATE = '2022-06-01'

# df = YahooDownloader(
#     start_date = TRAIN_START_DATE,
#     end_date = TEST_END_DATE,
#     ticker_list = config_tickers.DOW_30_TICKER
# ).fetch_data()
# df.to_csv('./data/df.csv', index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
df = pd.read_csv('./data/df.csv')

In [4]:
INDICATORS = [
    'macd',
    'rsi_30',
    'cci_30',
    'dx_30'
]

In [6]:
fe = FeatureEngineer(
    use_technical_indicator = True,
    tech_indicator_list = INDICATORS,
    use_turbulence = False,
    user_defined_feature = False
)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

Successfully added technical indicators


In [18]:
# Add covariance matrix as states
processed = processed.sort_values(['date', 'tic'], ignore_index=True)
processed.index = processed.date.factorize()[0]

cov_list = []
lookback = 252  # look back is one year

for i in range(lookback,len(processed.index.unique())):
  data_lookback = processed.loc[i-lookback:i,:]
  price_lookback = data_lookback.pivot_table(
    index = 'date',
    columns = 'tic',
    values = 'close'
  )
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({
  'date':processed.date.unique()[lookback:],
  'cov_list': cov_list
})
processed = processed.merge(df_cov, on='date')
processed = processed.sort_values(['date','tic']).reset_index(drop=True)
processed.head()

processed.to_csv('./data/processed.csv', index=False)

In [ ]:
processed = pd.read_csv('./data/processed.csv')

In [9]:
# Set environment
stock_dimension = len(processed.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "transaction_cost_pct": 1,
}

Stock Dimension: 29, State Space: 175


In [17]:
env_train = StockPortfolioEnv(
    df = processed,
    **env_kwargs
)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
# Agent
rebalance_window = 63 # rebalance_window is the number of days to retrain the model
validation_window = 63 # validation_window is the number of days to do validation and trading (e.g. if validation_window=63, then both validation and trading period will be 63 days)

# ensemble_agent = DRLEnsembleAgent(
#     df=processed,
#     train_period=(TRAIN_START_DATE, TRAIN_END_DATE),
#     val_test_period=(TEST_START_DATE, TEST_END_DATE),
#     rebalance_window=rebalance_window, 
#     validation_window=validation_window, 
#     **env_kwargs
# )

agent = DRLAgent(
    env = env_train
)

In [ ]:
# Train
print("==============Model Training===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')
a2c_params_tuning = {
    'n_steps': 5, 
    'ent_coef': 0.005, 
    'learning_rate': 0.0003,
    'verbose': 0,
    'timesteps': 50000,
}
model_a2c = agent.train_A2C(
    model_name = "A2C_{}".format(now),
    model_params = a2c_params_tuning
) 

In [ ]:
# Trade
# trade = data_split(df, '2019-01-01', '2020-12-01')

# env_trade, obs_trade = env_setup.create_env_trading(
#     data = trade,
#     env_class = StockPortfolioEnv
# ) 

df_daily_return, df_actions = DRLAgent.DRL_prediction(
    model=model_a2c,
    test_data = trade,
    test_env = env_trade,
    test_obs = obs_trade
)